# Dubai — Quantum accurate bond inference and partial charge calculations

See [the tutorial](/Tutorials/ligand_bond_inference_and_partial_charge_calculation.ipynb).

In [ ]:
import os
import json
from pathlib import Path

import requests
import rush

# Define our project information
DESCRIPTION = "quantum-bond-inference-notebook"
TAGS = ["rush-py", "dubai", "convert"]
WORK_DIR = Path.home() / "qdx" / "dubai-rush-py-quickstart"

# |hide_line
if WORK_DIR.exists():
    # |hide_line
    client = rush.Provider(workspace=WORK_DIR)
    # |hide_line
    await client.nuke(remote=False)

os.makedirs(WORK_DIR, exist_ok=True)

# Get our client, for calling modules and using the rush API
# - edit in your access token and API URL or set the RUSH_TOKEN and RUSH_URL environment variables
client = await rush.build_provider_with_functions(
    workspace=WORK_DIR,
    batch_tags=TAGS,
)

# | hide_line
client = await rush.build_provider_with_functions(
    workspace=WORK_DIR, batch_tags=TAGS, restore_by_default=True
)

# Convert aspirin to a QDXF file so we can use it for this demo
SMILES_STRING = "CC(=O)OC1=CC=CC=C1C(=O)O"
SDF_LINK = (
    f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{SMILES_STRING}/record/SDF?record_type=3d"
)

file_path = f"{WORK_DIR}/aspirin.sdf"
with open(file_path, "wb") as f:
    f.write(requests.get(SDF_LINK).content)

# We need to specify storage > file size to ensure that we have allocated
# enough resources for the convert module
(ligand,) = await client.convert("SDF", Path(file_path), resources={"storage": 10000})

ligand = await ligand.get()

# Remove connectivity
EXPECTED_CONNECTIVITY = ligand[0]["topology"]["connectivity"]
ligand = ligand[0]
del ligand["topology"]["connectivity"]

ligand["topology"]["fragment_multiplicities"] = [1]

# Arguments
DUBAI_RESOURCES = {
    "gpus": 1,
    "storage": 1024_000,
    "walltime": 60,
}
LIGAND_FILEPATH = Path(f"{WORK_DIR}/aspirin.qdxf.json")
json.dump(ligand, open(LIGAND_FILEPATH, "w"))

# Run the inference
(ligand_with_bonds,) = await client.dubai(LIGAND_FILEPATH, resources=DUBAI_RESOURCES, target="NIX_SSH_3")

output_ligand = await ligand_with_bonds.get()

for expected_bond, outputted_bond in zip(EXPECTED_CONNECTIVITY, output_ligand["topology"]["connectivity"]):
    # Check start atoms are the same
    assert expected_bond[0] == outputted_bond[0]
    # Check ending atoms are the same
    assert expected_bond[1] == outputted_bond[1]
    # NB: we don't check the third item of the bond -- the bond type. This is
    # because Dubai accurately outputs ring bonds as a specific 'RINGBOND' type,
    # whereas SDF aspirin was interleaving single and double bonds.
print("Bond inference performed correctly!")